# Download bot byte code

The first step is downloading the byte code of each bot contract. We need it for clustering attackers.

<font color='red'>We included the compressed result in the repository, you only need to run the first two cells to decompress them.</font>

In order for this to work, you will need access to an Ethereum node.

In [1]:
import json
import os
import tarfile

from web3 import Web3

Consider installing rusty-rlp to improve pyrlp performance with a rust based backend


If we did this before, we can decompress the results. If not, create the directory to donwload the byte code.

<font color='red'>You do not need to continue after this cell.</font>

In [2]:
data_directory = os.path.join("..", "..", "..", "data")

file_name = "displacement_bot_bytecode"
directory_path = os.path.join(data_directory, file_name)
compressed_path = os.path.join(data_directory, "{}.tar.xz".format(file_name))

# there is already a compressed file
if os.path.exists(compressed_path):
    with tarfile.open(compressed_path, "r:xz", encoding="utf-8") as compressed_file:
        compressed_file.extractall(data_directory)
# there is not compressed file so create the directory if needed
elif not os.path.exists(directory_path):
    os.makedirs(directory_path)

Uncomment the line for the right web3 provider (websocket or http) and fill the host and port.

In [ ]:
# server = Web3(Web3.WebsocketProvider("ws://host:port"))

# server = Web3(Web3.HTTPProvider("http://host:port"))

Decompress the results file (if needed).

In [4]:
results_file_path = os.path.join(data_directory, "displacement_results.json")

if not os.path.exists(results_file_path):
    with tarfile.open(os.path.join(data_directory, "displacement_results.tar.xz"), "r:xz", encoding="utf-8") as compressed_file:
        compressed_file.extract("displacement_results.json", data_directory)

Read the results one line at a time. Each line is json encoded.

In [5]:
with open(results_file_path, "r", encoding="utf-8") as results_file:
    results = []
    line = results_file.readline().strip()
    while line != "":
        results.append(json.loads(line))
        line = results_file.readline().strip()

If a contract was destroyed we cannot fetch the bytecode. Since the code does not change (unless destroyed), we will request the code from an arbitrary block (the smallest) where we know that the contract was not yet destroyed.

In [6]:
first_block_by_bots = {}

for result in results:
    # ignore the bad results
    if result["attacker_transaction"]["to"] == result["victim_transaction"]["to"]:
        continue
    
    attacker_transaction = result["attacker_transaction"]

    bot = attacker_transaction["to"]
    block = attacker_transaction["blockNumber"]
    
    # if we do not have block for this bot we keep it
    if bot not in first_block_by_bots:
        first_block_by_bots[bot] = block
    # if not we keep it if it is smaller
    elif block < first_block_by_bots[bot]:
        first_block_by_bots[bot] = block

Request the byte code for each bot. We use {address}-{block}.bin as a convention for the file names.

In [7]:
for bot, first_block in first_block_by_bots.items():
    code_file_path = os.path.join(directory_path, "{}-{}.bin".format(bot, first_block))
    
    # skip if already downloaded
    if os.path.exists(code_file_path):
        continue
        
    # I had to transform the address because of an error
    # maybe the address was transformed to lower case
    # see: https://stackoverflow.com/questions/57335994/trying-to-call-balanceof-function-in-ethereum-using-web3-py-library-but-getting
    bot = Web3.toChecksumAddress(bot)
    
    code = server.eth.getCode(bot, block_identifier=first_block)
    
    with open(code_file_path, "wb") as code_file:
        code_file.write(code)

Compress the results.

In [8]:
with tarfile.open(compressed_path, "w:xz") as compressed_file:
    compressed_file.add(directory_path, arcname=os.path.basename(file_name))